In [29]:
import pandas as pd
import numpy as np
import os
import zipfile as zf
import shutil
import re
import seaborn as sns
import random
import keras
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.layers import Dense,GlobalAveragePooling2D
from keras.layers import Dropout
from keras.applications import MobileNet
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Input
from keras.optimizers import Adam
from sklearn.metrics import  classification_report

In [ ]:
!gdown 1M_u-5UjskP8YbuhagbBdCiNYvTP69swO

In [ ]:
files = zf.ZipFile("/content/trash_images rev 3.zip",'r')
files.extractall()
files.close()
os.listdir(os.path.join(os.getcwd(),"trash_images"))

In [4]:
def split_indices(folder,seed1,seed2):
    n = len(os.listdir(folder))
    full_set = list(range(1,n+1))

    ## train indices
    random.seed(seed1)
    train = random.sample(list(range(1,n+1)),int(.7*n))

    ## temp
    remain = list(set(full_set)-set(train))

    ## separate remaining into validation and test
    random.seed(seed2)
    valid = random.sample(remain,int(.5*len(remain)))
    test = list(set(remain)-set(valid))

    return(train,valid,test)

def get_names(waste_type, indices):
    file_names = []
    for i in indices:
        file_name = f"{waste_type} ({i}).jpg"  # Format nama file yang diinginkan
        file_names.append(file_name)
    return (file_names)

def move_files(source_files,destination_folder):
    for file in source_files:
        shutil.move(file,destination_folder)

In [9]:
# Create destination folders for data subset and waste type
subsets = ['train', 'test']
waste_types = ['glass', 'paper', 'trash']

# Ensure destination folders exist
for subset in subsets:
    for waste_type in waste_types:
        folder = os.path.join('data', subset, waste_type)
        if not os.path.exists(folder):
            os.makedirs(folder)

In [10]:
# Move files to destination folders for each waste type
for waste_type in waste_types:
    source_folder = os.path.join('trash_images', waste_type)
    train_ind, test_ind = split_indices(source_folder, 1)

    # Move source files to train
    train_names = get_names(waste_type, train_ind)
    train_source_files = [os.path.join(source_folder, name) for name in train_names]
    train_dest = os.path.join("data/train", waste_type)
    move_files(train_source_files, train_dest)

    # Move source files to test
    test_names = get_names(waste_type, test_ind)
    test_source_files = [os.path.join(source_folder, name) for name in test_names]
    test_dest = os.path.join("data/test", waste_type)
    move_files(test_source_files, test_dest)

File not found: trash_images/trash/trash (763).jpg


In [12]:
base_model=MobileNet(weights='imagenet',include_top=False,
                     input_shape=(224, 224, 3))

17225924/17225924 [==============================] - 0s 0us/step


In [ ]:
base_model.summary()

In [18]:
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(6, activation='softmax')
])

In [19]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
len(model.layers)

In [21]:
for layer in model.layers[:20]:
    layer.trainable=False
for layer in model.layers[20:]:
    layer.trainable=True

In [ ]:
model.summary()

In [24]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   fill_mode='nearest',
                                   horizontal_flip=True,
                                   )

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [ ]:
from imutils import paths
train_dir = 'data/train'
validation_dir = 'data/valid/'
test_dir = 'data/test/'
totalTrain = len(list(paths.list_images(train_dir)))
totalVal = len(list(paths.list_images(validation_dir)))
totalTest = len(list(paths.list_images(test_dir)))
print("Total Training: ", totalTrain)
print("Total Validation: ",totalVal)
print("Total test: ", totalTest)

In [26]:
BATCH_SIZE = 16
TARGET_SIZE = (224, 224)

In [27]:
train_generator = train_datagen.flow_from_directory(train_dir,
                                                   batch_size=BATCH_SIZE,
                                                   class_mode='categorical',
                                                    color_mode='rgb',
                                                    shuffle=True,
                                                   target_size=TARGET_SIZE)

validation_generator =  test_datagen.flow_from_directory(validation_dir,
                                                   batch_size=BATCH_SIZE,
                                                   color_mode='rgb',
                                                   class_mode='categorical',
                                                   shuffle=False,
                                                   target_size=TARGET_SIZE)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    class_mode="categorical",
    color_mode='rgb',
    target_size=TARGET_SIZE,
    shuffle=False,
    batch_size=BATCH_SIZE)

Found 3361 images belonging to 6 classes.
Found 559 images belonging to 6 classes.
Found 857 images belonging to 6 classes.


In [ ]:
model.fit_generator(
    train_generator,
    steps_per_epoch=totalTrain // BATCH_SIZE,
    validation_data=validation_generator,
    validation_steps=totalVal // BATCH_SIZE,
    epochs=50)

Evaluasi

In [ ]:
import matplotlib.pyplot as plt
# plot the training loss and accuracy
N = np.arange(0, 50)
plt.style.use("ggplot")
plt.figure()
plt.plot(N, H.history["loss"], label="train_loss")
plt.plot(N, H.history["val_loss"], label="val_loss")
plt.title("Training Loss and Accuracy on CIFAR-10")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()
plt.figure()
plt.plot(N, H.history["accuracy"], label="train_acc")
plt.plot(N, H.history["val_accuracy"], label="val_accuracy")
plt.title("Training Loss and Accuracy on CIFAR-10")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()